In [3]:
import pandas as pd
from main import Click_house_cloud
import ast
import nltk
from nltk.stem.porter import PorterStemmer
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
client = Click_house_cloud()

In [5]:
Database = "Create Database IF NOT EXISTS Movies"
Use_db = "Use Movies"
result_db = client.command(Database)
result_use = client.command(Use_db)

In [6]:
query = "SELECT * FROM Movies.Final;"

# Execute the query and fetch the data

Movies_Combined_final = pd.DataFrame(client.query(query).result_rows,columns=["Movie_ID","Title","Tags"])


In [7]:
CV = CountVectorizer(max_features=5000,stop_words='english')

In [8]:
Vector = CV.fit_transform(Movies_Combined_final['Tags']).toarray()

In [9]:
PS = PorterStemmer()

In [10]:
def stem(text):
    L = []
    for i in text.split():
        L.append(PS.stem(i))
    return " ".join(L) 

In [11]:
Movies_Combined_final['Tags'] = Movies_Combined_final['Tags'].apply(stem)

In [12]:
Similarity = cosine_similarity(Vector)

In [13]:
def recommend(Movie):
    Movie_index = Movies_Combined_final[Movies_Combined_final['Title'] == Movie].index[0]
    distance = Similarity[Movie_index]
    Movie_List = sorted(list(enumerate(distance)),reverse=True,key=lambda x : x[1])[1:6]

    for i in Movie_List:
        print(Movies_Combined_final.iloc[i[0]].Title)

In [36]:
recommend("Avatar")

Titan A.E.
Small Soldiers
Independence Day
Aliens vs Predator: Requiem
Ender's Game


In [14]:
Movies_Combined_final.to_pickle('Movie_DF.pkl')

In [15]:
pickle.dump(Similarity,open('similarity.pkl','wb'))